In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import warnings
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import Image
warnings.filterwarnings('ignore')

In [27]:
train_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("compete.csv")

In [28]:
train_df = train_df.drop(['is_host_login', 'num_outbound_cmds'], axis=1);
test_df = test_df.drop(['is_host_login', 'num_outbound_cmds'], axis=1);

In [29]:
train_df = pd.get_dummies(train_df, columns=['protocol_type'])
test_df = pd.get_dummies(test_df, columns=['protocol_type'])

In [30]:
from sklearn import preprocessing

cat_cols = ['service', 'flag']
for col in cat_cols:
    if col in train_df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(list(train_df[col].astype(str).values) + list(test_df[col].astype(str).values))
        train_df[col] = le.transform(list(train_df[col].astype(str).values))
        test_df[col] = le.transform(list(test_df[col].astype(str).values))   

In [31]:
numerical_features = list(train_df.columns[train_df.dtypes != object].values[:-1])
categorical_features = list(train_df.columns[train_df.dtypes == object].values)

corr_table = train_df.corr()
triu = corr_table.where(np.triu(np.ones(corr_table.shape) ,k=1).astype(np.bool))
to_drop = [feat for feat in triu.columns if any(triu[feat] > 0.95)]

train_df = train_df.drop(to_drop, axis=1)

for feat in to_drop:
    if feat in categorical_features:
        categorical_features.remove(feat)
    else:
        numerical_features.remove(feat)

print(f'\nFeatures dropped: {to_drop}')
# plt.figure(figsize=(50, 30))
# _ = sns.heatmap(corr_table, annot=True, fmt='.2f')


Features dropped: ['num_root', 'srv_serror_rate', 'srv_rerror_rate', 'dst_host_same_srv_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'protocol_type_icmp']


In [32]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb

X = train_df.drop('class', axis=1)
y = train_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [33]:
lgb_model = lgb.LGBMClassifier(bagging_fraction=0.8446682014322044, feature_fraction= 0.35294922284424557,
 learning_rate= 0.5683390979599586,
 max_bin= 21,
 max_depth= 28,
 min_data_in_leaf= 77,
 min_sum_hessian_in_leaf= 0.2693920974409014,
 num_leaves= 35,
 subsample= 0.2095972365619459,
 objective= 'binary',
 metric= 'auc',
 is_unbalance = True,
 boost_from_average = False)
lgb_model.fit(X_train, y_train)
pred1 = lgb_model.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.35294922284424557, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.35294922284424557
[LightGBM] [Warning] min_data_in_leaf is set=77, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=77
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.2693920974409014, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.2693920974409014
[LightGBM] [Warning] bagging_fraction is set=0.8446682014322044, subsample=0.2095972365619459 will be ignored. Current value: bagging_fraction=0.8446682014322044


In [34]:
from sklearn.model_selection import KFold

kfold = 15
skf = KFold(n_splits=kfold)
pred2 = pd.DataFrame()
pred2['class'] = np.zeros(len(X_test))

In [23]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

#Instantiate CatBoostClassifier
cbc = CatBoostClassifier()

#create the grid
grid = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300], 'learning_rate': [0.005, 0.001, 0.05, 0.01, 0.1]}

#Instantiate GridSearchCV
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring ='accuracy', cv = 5)

#fit the model
gscv.fit(X,y)

#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0:	learn: 0.6726406	total: 47.8ms	remaining: 4.73s
1:	learn: 0.6579079	total: 80.2ms	remaining: 3.93s
2:	learn: 0.6401551	total: 109ms	remaining: 3.54s
3:	learn: 0.6218689	total: 140ms	remaining: 3.35s
4:	learn: 0.6091719	total: 169ms	remaining: 3.21s
5:	learn: 0.5919561	total: 199ms	remaining: 3.11s
6:	learn: 0.5800906	total: 229ms	remaining: 3.04s
7:	learn: 0.5623245	total: 257ms	remaining: 2.95s
8:	learn: 0.5464373	total: 289ms	remaining: 2.92s
9:	learn: 0.5350705	total: 319ms	remaining: 2.88s
10:	learn: 0.5233801	total: 348ms	remaining: 2.81s
11:	learn: 0.5084946	total: 377ms	remaining: 2.77s
12:	learn: 0.4989954	total: 406ms	remaining: 2.71s
13:	learn: 0.4891586	total: 434ms	remaining: 2.66s
14:	learn: 0.4742057	total: 466ms	remaining: 2.64s
15:	learn: 0.4615273	total: 497ms	remaining: 2.61s
16:	learn: 0.4483782	total: 529ms	remaining: 2.58s
17:	learn: 0.4392750	total: 560ms	remaining: 2.55s
18:	learn: 0.4267576	total: 592ms	remaining: 2.52s
19:	learn: 0.4186653	total: 621ms	remai

64:	learn: 0.1703292	total: 2.06s	remaining: 1.11s
65:	learn: 0.1672107	total: 2.09s	remaining: 1.08s
66:	learn: 0.1649509	total: 2.12s	remaining: 1.04s
67:	learn: 0.1613441	total: 2.14s	remaining: 1.01s
68:	learn: 0.1592080	total: 2.17s	remaining: 976ms
69:	learn: 0.1571699	total: 2.2s	remaining: 943ms
70:	learn: 0.1533167	total: 2.23s	remaining: 912ms
71:	learn: 0.1498581	total: 2.27s	remaining: 881ms
72:	learn: 0.1464957	total: 2.29s	remaining: 849ms
73:	learn: 0.1444064	total: 2.33s	remaining: 817ms
74:	learn: 0.1416849	total: 2.35s	remaining: 785ms
75:	learn: 0.1396544	total: 2.38s	remaining: 753ms
76:	learn: 0.1383005	total: 2.41s	remaining: 720ms
77:	learn: 0.1368166	total: 2.44s	remaining: 687ms
78:	learn: 0.1342750	total: 2.47s	remaining: 656ms
79:	learn: 0.1323265	total: 2.49s	remaining: 623ms
80:	learn: 0.1298671	total: 2.52s	remaining: 592ms
81:	learn: 0.1283432	total: 2.55s	remaining: 559ms
82:	learn: 0.1270266	total: 2.58s	remaining: 528ms
83:	learn: 0.1250727	total: 2.6s

28:	learn: 0.3306468	total: 878ms	remaining: 2.15s
29:	learn: 0.3245983	total: 906ms	remaining: 2.11s
30:	learn: 0.3186710	total: 935ms	remaining: 2.08s
31:	learn: 0.3129125	total: 963ms	remaining: 2.05s
32:	learn: 0.3074693	total: 993ms	remaining: 2.02s
33:	learn: 0.2994552	total: 1.02s	remaining: 1.98s
34:	learn: 0.2939431	total: 1.05s	remaining: 1.95s
35:	learn: 0.2885893	total: 1.08s	remaining: 1.92s
36:	learn: 0.2832300	total: 1.11s	remaining: 1.88s
37:	learn: 0.2785650	total: 1.13s	remaining: 1.85s
38:	learn: 0.2717750	total: 1.16s	remaining: 1.81s
39:	learn: 0.2674867	total: 1.19s	remaining: 1.78s
40:	learn: 0.2605307	total: 1.21s	remaining: 1.75s
41:	learn: 0.2562356	total: 1.24s	remaining: 1.72s
42:	learn: 0.2521801	total: 1.27s	remaining: 1.69s
43:	learn: 0.2482156	total: 1.3s	remaining: 1.65s
44:	learn: 0.2447067	total: 1.32s	remaining: 1.62s
45:	learn: 0.2407539	total: 1.35s	remaining: 1.59s
46:	learn: 0.2366833	total: 1.38s	remaining: 1.56s
47:	learn: 0.2326626	total: 1.41

90:	learn: 0.1133297	total: 2.84s	remaining: 281ms
91:	learn: 0.1117086	total: 2.87s	remaining: 249ms
92:	learn: 0.1101120	total: 2.89s	remaining: 218ms
93:	learn: 0.1084893	total: 2.92s	remaining: 186ms
94:	learn: 0.1071203	total: 2.95s	remaining: 155ms
95:	learn: 0.1061258	total: 2.97s	remaining: 124ms
96:	learn: 0.1051127	total: 3s	remaining: 92.7ms
97:	learn: 0.1035625	total: 3.03s	remaining: 61.8ms
98:	learn: 0.1019609	total: 3.06s	remaining: 30.9ms
99:	learn: 0.1011877	total: 3.08s	remaining: 0us
0:	learn: 0.6726406	total: 37.4ms	remaining: 7.45s
1:	learn: 0.6579079	total: 69.2ms	remaining: 6.85s
2:	learn: 0.6401551	total: 99.6ms	remaining: 6.54s
3:	learn: 0.6218689	total: 130ms	remaining: 6.38s
4:	learn: 0.6091719	total: 161ms	remaining: 6.29s
5:	learn: 0.5919561	total: 193ms	remaining: 6.25s
6:	learn: 0.5800906	total: 223ms	remaining: 6.14s
7:	learn: 0.5623245	total: 254ms	remaining: 6.1s
8:	learn: 0.5464373	total: 283ms	remaining: 6s
9:	learn: 0.5350705	total: 312ms	remaining:

151:	learn: 0.0623875	total: 4.35s	remaining: 1.37s
152:	learn: 0.0619122	total: 4.38s	remaining: 1.34s
153:	learn: 0.0615786	total: 4.4s	remaining: 1.31s
154:	learn: 0.0610083	total: 4.43s	remaining: 1.29s
155:	learn: 0.0606215	total: 4.46s	remaining: 1.26s
156:	learn: 0.0601882	total: 4.48s	remaining: 1.23s
157:	learn: 0.0598554	total: 4.51s	remaining: 1.2s
158:	learn: 0.0588470	total: 4.54s	remaining: 1.17s
159:	learn: 0.0584396	total: 4.57s	remaining: 1.14s
160:	learn: 0.0574790	total: 4.6s	remaining: 1.11s
161:	learn: 0.0569682	total: 4.63s	remaining: 1.08s
162:	learn: 0.0563351	total: 4.65s	remaining: 1.06s
163:	learn: 0.0557122	total: 4.68s	remaining: 1.03s
164:	learn: 0.0553099	total: 4.71s	remaining: 999ms
165:	learn: 0.0545519	total: 4.74s	remaining: 970ms
166:	learn: 0.0543001	total: 4.76s	remaining: 942ms
167:	learn: 0.0539484	total: 4.79s	remaining: 913ms
168:	learn: 0.0537078	total: 4.82s	remaining: 883ms
169:	learn: 0.0534777	total: 4.84s	remaining: 854ms
170:	learn: 0.0

...

140:	learn: 0.0020957	total: 4.12s	remaining: 4.65s
141:	learn: 0.0020956	total: 4.15s	remaining: 4.62s
142:	learn: 0.0020956	total: 4.17s	remaining: 4.58s
143:	learn: 0.0020956	total: 4.2s	remaining: 4.55s
144:	learn: 0.0020956	total: 4.22s	remaining: 4.51s
145:	learn: 0.0020956	total: 4.24s	remaining: 4.48s
146:	learn: 0.0020956	total: 4.27s	remaining: 4.44s
147:	learn: 0.0020956	total: 4.29s	remaining: 4.41s
148:	learn: 0.0020956	total: 4.32s	remaining: 4.38s
149:	learn: 0.0020956	total: 4.35s	remaining: 4.35s
150:	learn: 0.0020956	total: 4.37s	remaining: 4.32s
151:	learn: 0.0020955	total: 4.4s	remaining: 4.28s
152:	learn: 0.0020954	total: 4.42s	remaining: 4.25s
153:	learn: 0.0020953	total: 4.45s	remaining: 4.21s
154:	learn: 0.0020953	total: 4.47s	remaining: 4.18s
155:	learn: 0.0020952	total: 4.5s	remaining: 4.15s
156:	learn: 0.0020952	total: 4.52s	remaining: 4.12s
157:	learn: 0.0020952	total: 4.55s	remaining: 4.09s
158:	learn: 0.0020952	total: 4.57s	remaining: 4.05s
159:	learn: 0.0

0:	learn: 0.3785168	total: 39.4ms	remaining: 11.8s
1:	learn: 0.2126716	total: 75.6ms	remaining: 11.3s
2:	learn: 0.1233301	total: 110ms	remaining: 10.9s
3:	learn: 0.0975028	total: 142ms	remaining: 10.5s
4:	learn: 0.0624499	total: 174ms	remaining: 10.2s
5:	learn: 0.0501245	total: 205ms	remaining: 10s
6:	learn: 0.0401689	total: 234ms	remaining: 9.79s
7:	learn: 0.0350960	total: 264ms	remaining: 9.63s
8:	learn: 0.0319788	total: 294ms	remaining: 9.49s
9:	learn: 0.0275922	total: 325ms	remaining: 9.42s
10:	learn: 0.0212807	total: 358ms	remaining: 9.4s
11:	learn: 0.0190444	total: 390ms	remaining: 9.35s
12:	learn: 0.0169057	total: 421ms	remaining: 9.29s
13:	learn: 0.0139777	total: 456ms	remaining: 9.31s
14:	learn: 0.0128398	total: 488ms	remaining: 9.28s
15:	learn: 0.0117699	total: 524ms	remaining: 9.3s
16:	learn: 0.0106211	total: 556ms	remaining: 9.25s
17:	learn: 0.0100364	total: 587ms	remaining: 9.19s
18:	learn: 0.0097838	total: 618ms	remaining: 9.15s
19:	learn: 0.0091031	total: 654ms	remaining

164:	learn: 0.0015989	total: 4.91s	remaining: 4.02s
165:	learn: 0.0015988	total: 4.94s	remaining: 3.99s
166:	learn: 0.0015987	total: 4.96s	remaining: 3.95s
167:	learn: 0.0015986	total: 4.98s	remaining: 3.92s
168:	learn: 0.0015985	total: 5.01s	remaining: 3.88s
169:	learn: 0.0015985	total: 5.03s	remaining: 3.85s
170:	learn: 0.0015984	total: 5.06s	remaining: 3.81s
171:	learn: 0.0015983	total: 5.08s	remaining: 3.78s
172:	learn: 0.0015978	total: 5.11s	remaining: 3.75s
173:	learn: 0.0015974	total: 5.13s	remaining: 3.72s
174:	learn: 0.0015973	total: 5.16s	remaining: 3.68s
175:	learn: 0.0015973	total: 5.18s	remaining: 3.65s
176:	learn: 0.0015973	total: 5.2s	remaining: 3.62s
177:	learn: 0.0015971	total: 5.23s	remaining: 3.58s
178:	learn: 0.0015971	total: 5.25s	remaining: 3.55s
179:	learn: 0.0015971	total: 5.28s	remaining: 3.52s
180:	learn: 0.0015970	total: 5.3s	remaining: 3.48s
181:	learn: 0.0015970	total: 5.33s	remaining: 3.45s
182:	learn: 0.0015970	total: 5.35s	remaining: 3.42s
183:	learn: 0.

27:	learn: 0.0142689	total: 1.11s	remaining: 10.8s
28:	learn: 0.0137000	total: 1.16s	remaining: 10.8s
29:	learn: 0.0132047	total: 1.19s	remaining: 10.7s
30:	learn: 0.0125461	total: 1.23s	remaining: 10.7s
31:	learn: 0.0120803	total: 1.27s	remaining: 10.6s
32:	learn: 0.0115717	total: 1.31s	remaining: 10.6s
33:	learn: 0.0108768	total: 1.35s	remaining: 10.6s
34:	learn: 0.0106241	total: 1.39s	remaining: 10.5s
35:	learn: 0.0102973	total: 1.43s	remaining: 10.5s
36:	learn: 0.0100448	total: 1.46s	remaining: 10.4s
37:	learn: 0.0098444	total: 1.5s	remaining: 10.4s
38:	learn: 0.0094835	total: 1.54s	remaining: 10.3s
39:	learn: 0.0091439	total: 1.58s	remaining: 10.2s
40:	learn: 0.0087022	total: 1.62s	remaining: 10.2s
41:	learn: 0.0082679	total: 1.66s	remaining: 10.2s
42:	learn: 0.0079016	total: 1.7s	remaining: 10.1s
43:	learn: 0.0077998	total: 1.73s	remaining: 10.1s
44:	learn: 0.0076025	total: 1.77s	remaining: 10s
45:	learn: 0.0074448	total: 1.81s	remaining: 9.98s
46:	learn: 0.0074013	total: 1.85s	r

187:	learn: 0.0017904	total: 7.3s	remaining: 4.35s
188:	learn: 0.0017904	total: 7.36s	remaining: 4.32s
189:	learn: 0.0017904	total: 7.39s	remaining: 4.28s
190:	learn: 0.0017870	total: 7.42s	remaining: 4.24s
191:	learn: 0.0017870	total: 7.45s	remaining: 4.19s
192:	learn: 0.0017870	total: 7.48s	remaining: 4.15s
193:	learn: 0.0017870	total: 7.51s	remaining: 4.11s
194:	learn: 0.0017870	total: 7.54s	remaining: 4.06s
195:	learn: 0.0017870	total: 7.57s	remaining: 4.02s
196:	learn: 0.0017870	total: 7.6s	remaining: 3.97s
197:	learn: 0.0017767	total: 7.64s	remaining: 3.93s
198:	learn: 0.0017681	total: 7.67s	remaining: 3.89s
199:	learn: 0.0017472	total: 7.71s	remaining: 3.86s
200:	learn: 0.0017472	total: 7.75s	remaining: 3.82s
201:	learn: 0.0017471	total: 7.78s	remaining: 3.77s
202:	learn: 0.0017415	total: 7.83s	remaining: 3.74s
203:	learn: 0.0017415	total: 7.86s	remaining: 3.7s
204:	learn: 0.0017328	total: 7.9s	remaining: 3.66s
205:	learn: 0.0017328	total: 7.93s	remaining: 3.62s
206:	learn: 0.00

In [35]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(max_depth=5, n_estimators=300, learning_rate=0.05)
cat_model.fit(X_train, y_train)
pred3 = cat_model.predict(X_test)

0:	learn: 0.5158565	total: 45.2ms	remaining: 13.5s
1:	learn: 0.3703600	total: 83.3ms	remaining: 12.4s
2:	learn: 0.2809533	total: 120ms	remaining: 11.9s
3:	learn: 0.2338677	total: 154ms	remaining: 11.4s
4:	learn: 0.1723054	total: 188ms	remaining: 11.1s
5:	learn: 0.1461984	total: 221ms	remaining: 10.8s
6:	learn: 0.1257880	total: 254ms	remaining: 10.6s
7:	learn: 0.0927124	total: 289ms	remaining: 10.5s
8:	learn: 0.0731791	total: 323ms	remaining: 10.4s
9:	learn: 0.0657256	total: 353ms	remaining: 10.2s
10:	learn: 0.0536603	total: 387ms	remaining: 10.2s
11:	learn: 0.0475329	total: 418ms	remaining: 10s
12:	learn: 0.0436230	total: 449ms	remaining: 9.91s
13:	learn: 0.0369975	total: 485ms	remaining: 9.91s
14:	learn: 0.0338483	total: 517ms	remaining: 9.83s
15:	learn: 0.0303022	total: 551ms	remaining: 9.78s
16:	learn: 0.0270631	total: 583ms	remaining: 9.71s
17:	learn: 0.0246251	total: 614ms	remaining: 9.62s
18:	learn: 0.0220169	total: 647ms	remaining: 9.57s
19:	learn: 0.0207223	total: 678ms	remaini

162:	learn: 0.0022998	total: 4.66s	remaining: 3.92s
163:	learn: 0.0022946	total: 4.69s	remaining: 3.89s
164:	learn: 0.0022845	total: 4.71s	remaining: 3.86s
165:	learn: 0.0022693	total: 4.75s	remaining: 3.83s
166:	learn: 0.0022693	total: 4.77s	remaining: 3.8s
167:	learn: 0.0022693	total: 4.79s	remaining: 3.77s
168:	learn: 0.0022376	total: 4.82s	remaining: 3.74s
169:	learn: 0.0022376	total: 4.84s	remaining: 3.71s
170:	learn: 0.0022220	total: 4.87s	remaining: 3.68s
171:	learn: 0.0022220	total: 4.89s	remaining: 3.64s
172:	learn: 0.0022219	total: 4.92s	remaining: 3.61s
173:	learn: 0.0022172	total: 4.94s	remaining: 3.58s
174:	learn: 0.0022172	total: 4.96s	remaining: 3.54s
175:	learn: 0.0022172	total: 4.99s	remaining: 3.51s
176:	learn: 0.0022172	total: 5.01s	remaining: 3.48s
177:	learn: 0.0022172	total: 5.03s	remaining: 3.45s
178:	learn: 0.0022171	total: 5.05s	remaining: 3.41s
179:	learn: 0.0022171	total: 5.07s	remaining: 3.38s
180:	learn: 0.0022171	total: 5.09s	remaining: 3.35s
181:	learn: 0

In [22]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001 )

random_search.fit(X, y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000001D18D214B30>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.02,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, mis...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None, silent=True,
                                           subsa

In [25]:
random_search.best_params_

{'subsample': 0.6,
 'min_child_weight': 1,
 'max_depth': 5,
 'gamma': 1.5,
 'colsample_bytree': 0.8}

In [36]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb_model = xgb.XGBClassifier(subsample= 0.6, min_child_weight = 1, max_depth = 5, gamma = 1.5, colsample_bytree = 0.8)

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    X_train_s, X_valid = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_s, y_valid = y_train.iloc[train_index], y_train.iloc[test_index]
    xgb_model.fit(X_train_s, y_train_s, eval_set=[(X_train_s, y_train_s), (X_valid, y_valid)], eval_metric='auc', early_stopping_rounds=100)
    vtest = xgb_model.predict(X_test)
    pred2['class'] += vtest/kfold

[0]	validation_0-auc:0.99851	validation_1-auc:0.99789
[1]	validation_0-auc:0.99983	validation_1-auc:0.99987
[2]	validation_0-auc:0.99993	validation_1-auc:0.99997
[3]	validation_0-auc:0.99997	validation_1-auc:0.99999
[4]	validation_0-auc:0.99998	validation_1-auc:1.00000
[5]	validation_0-auc:0.99998	validation_1-auc:1.00000
[6]	validation_0-auc:0.99998	validation_1-auc:1.00000
[7]	validation_0-auc:0.99999	validation_1-auc:1.00000
[8]	validation_0-auc:0.99999	validation_1-auc:1.00000
[9]	validation_0-auc:0.99999	validation_1-auc:1.00000
[10]	validation_0-auc:0.99999	validation_1-auc:1.00000
[11]	validation_0-auc:0.99999	validation_1-auc:1.00000
[12]	validation_0-auc:0.99999	validation_1-auc:1.00000
[13]	validation_0-auc:0.99999	validation_1-auc:1.00000
[14]	validation_0-auc:0.99999	validation_1-auc:1.00000
[15]	validation_0-auc:0.99999	validation_1-auc:1.00000
[16]	validation_0-auc:1.00000	validation_1-auc:1.00000
[17]	validation_0-auc:1.00000	validation_1-auc:1.00000
[18]	validation_0-au

[50]	validation_0-auc:1.00000	validation_1-auc:1.00000
[51]	validation_0-auc:1.00000	validation_1-auc:1.00000
[52]	validation_0-auc:1.00000	validation_1-auc:1.00000
[53]	validation_0-auc:1.00000	validation_1-auc:1.00000
[54]	validation_0-auc:1.00000	validation_1-auc:1.00000
[55]	validation_0-auc:1.00000	validation_1-auc:1.00000
[56]	validation_0-auc:1.00000	validation_1-auc:1.00000
[57]	validation_0-auc:1.00000	validation_1-auc:1.00000
[58]	validation_0-auc:1.00000	validation_1-auc:1.00000
[59]	validation_0-auc:1.00000	validation_1-auc:1.00000
[60]	validation_0-auc:1.00000	validation_1-auc:1.00000
[61]	validation_0-auc:1.00000	validation_1-auc:1.00000
[62]	validation_0-auc:1.00000	validation_1-auc:1.00000
[63]	validation_0-auc:1.00000	validation_1-auc:1.00000
[64]	validation_0-auc:1.00000	validation_1-auc:1.00000
[65]	validation_0-auc:1.00000	validation_1-auc:1.00000
[66]	validation_0-auc:1.00000	validation_1-auc:1.00000
[67]	validation_0-auc:1.00000	validation_1-auc:1.00000
[68]	valid

[0]	validation_0-auc:0.99844	validation_1-auc:0.99888
[1]	validation_0-auc:0.99982	validation_1-auc:0.99995
[2]	validation_0-auc:0.99993	validation_1-auc:0.99997
[3]	validation_0-auc:0.99997	validation_1-auc:0.99999
[4]	validation_0-auc:0.99998	validation_1-auc:0.99999
[5]	validation_0-auc:0.99998	validation_1-auc:1.00000
[6]	validation_0-auc:0.99998	validation_1-auc:0.99999
[7]	validation_0-auc:0.99998	validation_1-auc:1.00000
[8]	validation_0-auc:0.99999	validation_1-auc:1.00000
[9]	validation_0-auc:0.99999	validation_1-auc:1.00000
[10]	validation_0-auc:0.99999	validation_1-auc:1.00000
[11]	validation_0-auc:0.99999	validation_1-auc:1.00000
[12]	validation_0-auc:0.99999	validation_1-auc:1.00000
[13]	validation_0-auc:1.00000	validation_1-auc:1.00000
[14]	validation_0-auc:1.00000	validation_1-auc:1.00000
[15]	validation_0-auc:1.00000	validation_1-auc:1.00000
[16]	validation_0-auc:1.00000	validation_1-auc:1.00000
[17]	validation_0-auc:1.00000	validation_1-auc:1.00000
[18]	validation_0-au

[50]	validation_0-auc:1.00000	validation_1-auc:1.00000
[51]	validation_0-auc:1.00000	validation_1-auc:1.00000
[52]	validation_0-auc:1.00000	validation_1-auc:1.00000
[53]	validation_0-auc:1.00000	validation_1-auc:1.00000
[54]	validation_0-auc:1.00000	validation_1-auc:1.00000
[55]	validation_0-auc:1.00000	validation_1-auc:1.00000
[56]	validation_0-auc:1.00000	validation_1-auc:1.00000
[57]	validation_0-auc:1.00000	validation_1-auc:1.00000
[58]	validation_0-auc:1.00000	validation_1-auc:1.00000
[59]	validation_0-auc:1.00000	validation_1-auc:1.00000
[60]	validation_0-auc:1.00000	validation_1-auc:1.00000
[61]	validation_0-auc:1.00000	validation_1-auc:1.00000
[62]	validation_0-auc:1.00000	validation_1-auc:1.00000
[63]	validation_0-auc:1.00000	validation_1-auc:1.00000
[64]	validation_0-auc:1.00000	validation_1-auc:1.00000
[65]	validation_0-auc:1.00000	validation_1-auc:1.00000
[66]	validation_0-auc:1.00000	validation_1-auc:1.00000
[67]	validation_0-auc:1.00000	validation_1-auc:1.00000
[68]	valid

[0]	validation_0-auc:0.99856	validation_1-auc:0.99860
[1]	validation_0-auc:0.99988	validation_1-auc:0.99987
[2]	validation_0-auc:0.99993	validation_1-auc:0.99991
[3]	validation_0-auc:0.99996	validation_1-auc:0.99994
[4]	validation_0-auc:0.99996	validation_1-auc:0.99994
[5]	validation_0-auc:0.99996	validation_1-auc:0.99994
[6]	validation_0-auc:0.99997	validation_1-auc:0.99994
[7]	validation_0-auc:0.99997	validation_1-auc:0.99994
[8]	validation_0-auc:0.99998	validation_1-auc:0.99997
[9]	validation_0-auc:0.99998	validation_1-auc:0.99997
[10]	validation_0-auc:0.99999	validation_1-auc:0.99999
[11]	validation_0-auc:0.99999	validation_1-auc:0.99999
[12]	validation_0-auc:0.99999	validation_1-auc:0.99999
[13]	validation_0-auc:0.99999	validation_1-auc:0.99999
[14]	validation_0-auc:0.99999	validation_1-auc:0.99999
[15]	validation_0-auc:0.99999	validation_1-auc:0.99999
[16]	validation_0-auc:0.99999	validation_1-auc:0.99999
[17]	validation_0-auc:1.00000	validation_1-auc:1.00000
[18]	validation_0-au

[50]	validation_0-auc:1.00000	validation_1-auc:1.00000
[51]	validation_0-auc:1.00000	validation_1-auc:1.00000
[52]	validation_0-auc:1.00000	validation_1-auc:1.00000
[53]	validation_0-auc:1.00000	validation_1-auc:1.00000
[54]	validation_0-auc:1.00000	validation_1-auc:1.00000
[55]	validation_0-auc:1.00000	validation_1-auc:1.00000
[56]	validation_0-auc:1.00000	validation_1-auc:1.00000
[57]	validation_0-auc:1.00000	validation_1-auc:1.00000
[58]	validation_0-auc:1.00000	validation_1-auc:1.00000
[59]	validation_0-auc:1.00000	validation_1-auc:1.00000
[60]	validation_0-auc:1.00000	validation_1-auc:1.00000
[61]	validation_0-auc:1.00000	validation_1-auc:1.00000
[62]	validation_0-auc:1.00000	validation_1-auc:1.00000
[63]	validation_0-auc:1.00000	validation_1-auc:1.00000
[64]	validation_0-auc:1.00000	validation_1-auc:1.00000
[65]	validation_0-auc:1.00000	validation_1-auc:1.00000
[66]	validation_0-auc:1.00000	validation_1-auc:1.00000
[67]	validation_0-auc:1.00000	validation_1-auc:1.00000
[68]	valid

[0]	validation_0-auc:0.99852	validation_1-auc:0.99755
[1]	validation_0-auc:0.99984	validation_1-auc:0.99959
[2]	validation_0-auc:0.99994	validation_1-auc:0.99990
[3]	validation_0-auc:0.99997	validation_1-auc:0.99993
[4]	validation_0-auc:0.99998	validation_1-auc:0.99993
[5]	validation_0-auc:0.99998	validation_1-auc:0.99994
[6]	validation_0-auc:0.99999	validation_1-auc:0.99997
[7]	validation_0-auc:0.99999	validation_1-auc:0.99997
[8]	validation_0-auc:0.99999	validation_1-auc:0.99998
[9]	validation_0-auc:0.99999	validation_1-auc:0.99998
[10]	validation_0-auc:0.99999	validation_1-auc:0.99998
[11]	validation_0-auc:0.99999	validation_1-auc:0.99998
[12]	validation_0-auc:0.99999	validation_1-auc:0.99998
[13]	validation_0-auc:0.99999	validation_1-auc:0.99998
[14]	validation_0-auc:0.99999	validation_1-auc:0.99999
[15]	validation_0-auc:1.00000	validation_1-auc:0.99999
[16]	validation_0-auc:1.00000	validation_1-auc:0.99999
[17]	validation_0-auc:1.00000	validation_1-auc:0.99999
[18]	validation_0-au

[50]	validation_0-auc:1.00000	validation_1-auc:1.00000
[51]	validation_0-auc:1.00000	validation_1-auc:1.00000
[52]	validation_0-auc:1.00000	validation_1-auc:1.00000
[53]	validation_0-auc:1.00000	validation_1-auc:1.00000
[54]	validation_0-auc:1.00000	validation_1-auc:1.00000
[55]	validation_0-auc:1.00000	validation_1-auc:1.00000
[56]	validation_0-auc:1.00000	validation_1-auc:1.00000
[57]	validation_0-auc:1.00000	validation_1-auc:1.00000
[58]	validation_0-auc:1.00000	validation_1-auc:1.00000
[59]	validation_0-auc:1.00000	validation_1-auc:1.00000
[60]	validation_0-auc:1.00000	validation_1-auc:1.00000
[61]	validation_0-auc:1.00000	validation_1-auc:1.00000
[62]	validation_0-auc:1.00000	validation_1-auc:1.00000
[63]	validation_0-auc:1.00000	validation_1-auc:1.00000
[64]	validation_0-auc:1.00000	validation_1-auc:1.00000
[65]	validation_0-auc:1.00000	validation_1-auc:1.00000
[66]	validation_0-auc:1.00000	validation_1-auc:1.00000
[67]	validation_0-auc:1.00000	validation_1-auc:1.00000
[68]	valid

[0]	validation_0-auc:0.99852	validation_1-auc:0.99921
[1]	validation_0-auc:0.99989	validation_1-auc:0.99986
[2]	validation_0-auc:0.99993	validation_1-auc:0.99992
[3]	validation_0-auc:0.99996	validation_1-auc:0.99994
[4]	validation_0-auc:0.99997	validation_1-auc:0.99994
[5]	validation_0-auc:0.99998	validation_1-auc:0.99994
[6]	validation_0-auc:0.99999	validation_1-auc:0.99998
[7]	validation_0-auc:0.99999	validation_1-auc:0.99998
[8]	validation_0-auc:0.99999	validation_1-auc:0.99998
[9]	validation_0-auc:0.99999	validation_1-auc:0.99998
[10]	validation_0-auc:0.99999	validation_1-auc:0.99999
[11]	validation_0-auc:0.99999	validation_1-auc:0.99999
[12]	validation_0-auc:0.99999	validation_1-auc:0.99999
[13]	validation_0-auc:0.99999	validation_1-auc:0.99999
[14]	validation_0-auc:1.00000	validation_1-auc:0.99999
[15]	validation_0-auc:1.00000	validation_1-auc:0.99999
[16]	validation_0-auc:1.00000	validation_1-auc:1.00000
[17]	validation_0-auc:1.00000	validation_1-auc:1.00000
[18]	validation_0-au

[50]	validation_0-auc:1.00000	validation_1-auc:1.00000
[51]	validation_0-auc:1.00000	validation_1-auc:1.00000
[52]	validation_0-auc:1.00000	validation_1-auc:1.00000
[53]	validation_0-auc:1.00000	validation_1-auc:1.00000
[54]	validation_0-auc:1.00000	validation_1-auc:1.00000
[55]	validation_0-auc:1.00000	validation_1-auc:1.00000
[56]	validation_0-auc:1.00000	validation_1-auc:1.00000
[57]	validation_0-auc:1.00000	validation_1-auc:1.00000
[58]	validation_0-auc:1.00000	validation_1-auc:1.00000
[59]	validation_0-auc:1.00000	validation_1-auc:1.00000
[60]	validation_0-auc:1.00000	validation_1-auc:1.00000
[61]	validation_0-auc:1.00000	validation_1-auc:1.00000
[62]	validation_0-auc:1.00000	validation_1-auc:1.00000
[63]	validation_0-auc:1.00000	validation_1-auc:1.00000
[64]	validation_0-auc:1.00000	validation_1-auc:1.00000
[65]	validation_0-auc:1.00000	validation_1-auc:1.00000
[66]	validation_0-auc:1.00000	validation_1-auc:1.00000
[67]	validation_0-auc:1.00000	validation_1-auc:1.00000
[68]	valid

In [37]:
test_id = test_df.Id.values
test_df = test_df.drop("Id", axis=1)

In [38]:
test_df = test_df.drop(to_drop, axis=1)

In [39]:
test_pred_lgm = lgb_model.predict(test_df)
test_pred_xgb = xgb_model.predict(test_df)
test_pred_cat = cat_model.predict(test_df)

In [40]:
stack_valid = np.column_stack((pred1, pred2['class'], pred3))
test_pred = np.column_stack((test_pred_lgm, test_pred_xgb, test_pred_cat))

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              random_state=0)

distributions = dict(C=uniform(loc=0, scale=4),
                     penalty=['l2', 'l1'])

clf = RandomizedSearchCV(logistic, distributions, random_state=0)
search = clf.fit(X, y)
search.best_params_

{'C': 2.195254015709299, 'penalty': 'l1'}

In [42]:
from sklearn.linear_model import LogisticRegression

stack_model = LogisticRegression(solver='saga', tol=1e-2, max_iter=200, C = 2.195254015709299, penalty = 'l1')
stack_model.fit(stack_valid, y_test)
test_stack_model = stack_model.predict(test_pred)

In [43]:
submit = pd.DataFrame({'Id': test_id, 'class':test_stack_model})
submit.to_csv('test.csv', index=False)